<a href="https://colab.research.google.com/github/RobertoJM-comp-error/GithubProyectU/blob/main/An%C3%A1lisis_de_Circuitos_en_A_C_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Análisis senoidal en estado estable***

Este análisis se usa cuando las fuentes son senoidales. Se representan mediante fasores (números complejos que indican magnitud y ángulo).

Fórmulas básicas:

Fasor de tensión:  $ \tilde{V} = V_m \angle \theta $  o $ \tilde{V} = V_{rms} \angle \theta $

Impedancias: $ Z_R = R $, $ Z_L = j \omega L $, $ Z_C = \dfrac{1}{j \omega C} $

Ley de Ohm fasorial: $ \tilde{V} = \tilde{I} , Z $

Conversión: $ V_{rms} = \dfrac{V_m}{\sqrt{2}} $

---

Paso a paso (circuito RLC en serie):

1. Convertir la fuente a fasor: $ \tilde{V} = V_m \angle \phi $ o $ V_{rms} \angle \phi $.

2. Calcular $ \omega = 2 \pi f $ y las impedancias $ Z_R, Z_L, Z_C $.

3. Sumar: $ Z_{eq} = Z_R + Z_L + Z_C $.

4. Corriente fasorial: $ \tilde{I} = \tilde{V} / Z_{eq} $.

5. Caídas: $ \tilde{V}_R = \tilde{I} Z_R $, $ \tilde{V}_L = \tilde{I} Z_L $, $ \tilde{V}_C = \tilde{I} Z_C $.

6. En el tiempo: $ i(t) = I_m \cos(\omega t + \theta_i) $.

---


In [ ]:
import numpy as np

# --- Utilidades ---
def grados_a_radianes(g):
    return g * np.pi / 180.0

def radianes_a_grados(r):
    return r * 180.0 / np.pi

def modulo_y_angulo(z):
    """Devuelve (modulo, angulo_en_grados) de un número complejo."""
    return np.abs(z), radianes_a_grados(np.angle(z))

def vrms_a_vm(Vrms):
    """Convierte Vrms a Vm (valor de amplitud pico)."""
    return Vrms * np.sqrt(2.0)

def vm_a_vrms(Vm):
    """Convierte Vm (pico) a Vrms."""
    return Vm / np.sqrt(2.0)

def fasor_desde_vrms(Vrms, angulo_grados=0.0):
    """Crea un fasor (en Vrms) con ángulo en grados."""
    return Vrms * np.exp(1j * grados_a_radianes(angulo_grados))

# --- Impedancias (comentarios con fórmulas) ---
def impedancia_R(R):
    # Z_R = R
    return complex(R)

def impedancia_L(omega, L):
    # Z_L = j * omega * L
    return 1j * omega * L

def impedancia_C(omega, C):
    # Z_C = 1 / (j * omega * C) = -j/(omega*C)
    return 1.0 / (1j * omega * C)

# --- Resolución de circuitos ---
def resolver_serie(Vrms_fasor, lista_impedancias):
    """
    Resuelve un circuito en serie:
    Vrms_fasor: fasor de la fuente en Vrms (complejo)
    lista_impedancias: lista de impedancias complejas [Z1, Z2, ...]
    Devuelve: Z_total, I_fasor (Vrms), lista_caidas (fasores Vrms)
    """
    Z_total = sum(lista_impedancias)
    I_fasor = Vrms_fasor / Z_total
    caidas = [I_fasor * Z for Z in lista_impedancias]
    return Z_total, I_fasor, caidas

def resolver_paralelo(Vrms_fasor, nodos_impedancias):
    """
    Resuelve un circuito de elementos en paralelo alimentado por Vrms_fasor.
    nodos_impedancias: lista de impedancias conectadas en paralelo [Z1, Z2, ...]
    Devuelve: lista_corrientes (fasores), I_total
    """
    corrientes = [Vrms_fasor / Z for Z in nodos_impedancias]
    I_total = sum(corrientes)
    return corrientes, I_total

# --- Ejemplos con valores típicos (60 Hz, 120 Vrms) ---
def resolver_ejemplos():
    print("=== Análisis senoidal en estado estable — Ejemplos ===\n")

    # Ejemplo 1: Serie R-L-C alimentada por 120 Vrms ∠ 0° a 60 Hz
    print("Ejemplo 1: Serie R-L-C (Vrms = 120 ∠ 0°, f = 60 Hz)")
    f = 60.0
    omega = 2.0 * np.pi * f
    Vrms = 120.0
    V_fasor = fasor_desde_vrms(Vrms, 0.0)  # 120 ∠ 0
    R = 10.0           # ohm
    L = 50e-3          # 50 mH
    C = 50e-6          # 50 µF
    Zr = impedancia_R(R)
    Zl = impedancia_L(omega, L)
    Zc = impedancia_C(omega, C)
    Zt, I_fasor, caidas = resolver_serie(V_fasor, [Zr, Zl, Zc])
    modI, angI = modulo_y_angulo(I_fasor)
    print(f"Impedancia total Zt = {Zt:.4f} Ω")
    print(f"I (fasor Vrms) = {I_fasor:.6f} A -> |I| = {modI:.6f} A, ángulo = {angI:.2f}°")
    nombres = ["VR (R)", "VL (L)", "VC (C)"]
    for n, v in zip(nombres, caidas):
        m,a = modulo_y_angulo(v)
        print(f"  {n}: {v:.4f} V -> |V| = {m:.3f} V, ángulo = {a:.2f}°")
    print()

    # Ejemplo 2: Paralelo R || C alimentado por 120 Vrms ∠ 0° a 50 Hz (ejemplo diferente f)
    print("Ejemplo 2: Paralelo R || C (Vrms = 120 ∠ 0°, f = 50 Hz)")
    f2 = 50.0
    omega2 = 2.0 * np.pi * f2
    Vrms2 = 120.0
    V2 = fasor_desde_vrms(Vrms2, 0.0)
    Rpar = 200.0
    Cpar = 20e-6
    Zr2 = impedancia_R(Rpar)
    Zc2 = impedancia_C(omega2, Cpar)
    corrientes, Itot = resolver_paralelo(V2, [Zr2, Zc2])
    Ir, Ic = corrientes
    mr, ar = modulo_y_angulo(Ir)
    mc, ac = modulo_y_angulo(Ic)
    mt, at = modulo_y_angulo(Itot)
    print(f"Ir = {Ir:.6f} A -> |Ir| = {mr:.6f} A, ángulo = {ar:.2f}°")
    print(f"Ic = {Ic:.6f} A -> |Ic| = {mc:.6f} A, ángulo = {ac:.2f}°")
    print(f"I_total = {Itot:.6f} A -> |I| = {mt:.6f} A, ángulo = {at:.2f}°")
    print()

    # Ejemplo 3: Método corto (usar Vm y |Z|) — Vrms dado, Z dado
    print("Ejemplo 3: Método corto — Vrms dado y Z dado (Vrms = 40, Z = 11.88 + j4.47)")
    Vrms3 = 40.0
    Z3 = 11.88 + 1j * 4.47
    Vm3 = vrms_a_vm(Vrms3)  # Vm (pico)
    Im_max = Vm3 / np.abs(Z3)  # Im (amplitud pico) según tu método corto
    # Para potencia activa usando Im_max y Vm:
    # P = (Vm * Im / 2) * pf  (si conocemos el pf)
    pf_angle = -np.angle(Z3)  # si V referencia 0°, I angle = -angle(Z)
    pf = np.cos(pf_angle)
    P = (Vm3 * Im_max / 2.0) * pf
    print(f"Z = {Z3:.4f} Ω -> |Z| = {np.abs(Z3):.4f} Ω")
    print(f"Vm (pico) = {Vm3:.4f} V, Im (pico) = {Im_max:.4f} A, pf ≈ {pf:.4f}")
    print(f"Potencia activa aproximada P = {P:.4f} W (método corto)")
    print()

    # Ejemplo 4: Combinación serie-paralelo (cálculo de Z equivalente)
    print("Ejemplo 4: Combinación serie-paralelo")
    Z1 = 50 + 1j * 20
    Z2 = 30 - 1j * 10
    Zpar = 1.0 / (1.0 / Z1 + 1.0 / Z2)
    Zeq = Zpar + (10 - 1j * 5)
    mz, az = modulo_y_angulo(Zeq)
    print(f"Z1 = {Z1}, Z2 = {Z2}")
    print(f"Z paralelo (Z1 || Z2) = {Zpar:.4f} Ω")
    print(f"Z equivalente total = {Zeq:.4f} Ω -> |Z| = {mz:.4f} Ω, ang = {az:.2f}°")
    print()

    # Ejemplo 5: Fuente con desfase (v = 150 cos(377 t + 10°)), Z dado -> hallar I fasor (amplitud)
    print("Ejemplo 5: Fuente con desfase y Z dado")
    omega5 = 377.0
    Vm5 = 150.0  # amplitud pico
    # Convertimos a fasor de amplitud: Vm∠10°, pero trabajamos con Vrms para consistencia -> Vrms = Vm / sqrt(2)
    Vrms5 = vm_a_vrms(Vm5)
    Vphasor5 = fasor_desde_vrms(Vrms5, 10.0)  # Vrms ∠ 10°
    Z5 = 11.88 + 1j * 4.47
    Iphasor5 = Vphasor5 / Z5
    m5, a5 = modulo_y_angulo(Iphasor5)
    print(f"Fuente: v(t) = 150 cos(377 t + 10°) -> Vrms = {Vrms5:.3f} V ∠ 10°")
    print(f"Z = {Z5}")
    print(f"I (fasor Vrms) = {Iphasor5:.6f} A -> |I| = {m5:.6f} A, ángulo = {a5:.2f}°")
    print()

if __name__ == "__main__":
    resolver_ejemplos()

=== Análisis Senoidal en Estado Estable: 5 ejemplos ===
Ejemplo 1 — Serie R-L-C:
Z_total = 10.0000-34.2021j
I (fasor, Amplitud) = 1.3388+4.5790j => |I|=4.7707 A, angle=73.70°
V_drop elemento 1: 13.3881+45.7901j -> |V|=47.707 V, ang=73.70°
V_drop elemento 2: -86.3124+25.2360j -> |V|=89.926 V, ang=163.70°
V_drop elemento 3: 242.9243-71.0261j -> |V|=253.095 V, ang=-16.30°

Ejemplo 2 — Paralelo R||C:
Ir=0.8485+0.0000j, Ic=-0.0000+1.0663j, Itot=0.8485+1.0663j

Ejemplo 3 — Método corto (Vm desde Vrms):
Vrms=40.0 V -> Vm=56.569 V; |Z|=12.693; Im=4.457 A; pf=0.936; P=117.978 W

Ejemplo 4 — Combinación serie-paralelo:
Z_eq = 31.0769-6.3846j

Ejemplo 5 — Fuente con desfase y Z dada:
V = 150∠10° (ampl) -> I = 11.6150-2.1778j -> |I|=11.8174 A angle=-10.62°



# ***Análisis de nodos y de mallas (en dominio fasorial)***

Ambos métodos usan impedancias complejas.

Nodal: aplica la Ley de Corriente de Kirchhoff (KCL).
Matriz general: $ \mathbf{G} , \mathbf{V} = \mathbf{I} $ donde $ G_{ij} = \dfrac{1}{Z_{ij}} $.

Mallas: aplica la Ley de Voltaje de Kirchhoff (KVL).
Matriz general: $ \mathbf{Z} , \mathbf{I} = \mathbf{V} $.

---

Paso a paso (análisis nodal):

1. Identificar los nodos y elegir uno como referencia.

2. Escribir KCL: $\displaystyle \sum_k \dfrac{V_n - V_k}{Z_{nk}} = I_{inj,n}$.

3. Reescribir en forma matricial $\mathbf{G}\mathbf{V} = \mathbf{I}$.

4. Resolver $\mathbf{V} = \mathbf{G}^{-1}\mathbf{I}$.

5. Obtener corrientes y potencias según necesidad.

---


In [ ]:
import numpy as np

# --- Utilidades ---
def grados_a_radianes(g):
    return g * np.pi / 180.0

def radianes_a_grados(r):
    return r * 180.0 / np.pi

def modulo_y_angulo(z):
    return np.abs(z), radianes_a_grados(np.angle(z))

def fasor_desde_vrms(Vrms, angulo_grados=0.0):
    return Vrms * np.exp(1j * grados_a_radianes(angulo_grados))

# --- Impedancias ---
def impedancia_R(R):
    # Z_R = R
    return complex(R)

def impedancia_L(omega, L):
    # Z_L = j * omega * L
    return 1j * omega * L

def impedancia_C(omega, C):
    # Z_C = 1 / (j * omega * C)
    return 1.0 / (1j * omega * C)

# --- ANÁLISIS NODAL (matriz de admitancias) ---
def construir_matriz_nodal(num_nodos, lista_admitancias, inyecciones_corriente):
    """
    Construye la matriz G (admitancias) y vector I para el análisis nodal.
    - num_nodos: número de nodos (excluye tierra). Índices 0..num_nodos-1.
    - lista_admitancias: lista de tuplas (n1, n2, Y) con nodos (None para tierra) y admitancia Y (compleja).
      Ejemplo: (0,1,Y01) indica admitancia entre nodo 0 y nodo 1.
      (0, None, Y0g) indica admitancia entre nodo 0 y tierra.
    - inyecciones_corriente: lista de tuplas (nodo, I_inyectada) (fasor, en A rms).
    Devuelve: G (num_nodos x num_nodos), I (num_nodos)
    """
    G = np.zeros((num_nodos, num_nodos), dtype=complex)
    I = np.zeros((num_nodos,), dtype=complex)

    for n1, n2, Y in lista_admitancias:
        if n1 is not None:
            G[n1, n1] += Y
        if n2 is not None:
            G[n2, n2] += Y
        if n1 is not None and n2 is not None:
            G[n1, n2] -= Y
            G[n2, n1] -= Y

    for n, Iinj in inyecciones_corriente:
        I[n] += Iinj

    return G, I

def resolver_nodal(G, I):
    """
    Resuelve G · V = I para obtener tensiones nodales (fasores Vrms).
    Devuelve array V de tamaño num_nodos (complejo).
    """
    V = np.linalg.solve(G, I)
    return V

# --- ANÁLISIS DE MALLAS (matriz de impedancias) ---
def construir_matriz_mallas(Zm, fuentes_V):
    """
    Zm: lista o array cuadrado (n_mallas x n_mallas) con impedancias complejas.
    fuentes_V: vector (n_mallas) con voltajes equivalentes de malla (fasores Vrms).
    Devuelve Zm_matrix, V_vector como numpy arrays.
    """
    Zm_mat = np.array(Zm, dtype=complex)
    V_vec = np.array(fuentes_V, dtype=complex)
    return Zm_mat, V_vec

def resolver_mallas(Zm_mat, V_vec):
    """
    Resuelve Zm · I_m = V (sistema de mallas).
    Devuelve I_m (vector de corrientes de malla, fasores Vrms).
    """
    Im = np.linalg.solve(Zm_mat, V_vec)
    return Im

# --- EJEMPLOS (mix: nodal y mallas) ---
def resolver_ejemplos():
    print("=== Análisis de Nodos y Mallas — Ejemplos ===\n")

    # Parámetros comunes
    f = 60.0
    omega = 2.0 * np.pi * f

    # ---------- EJEMPLO NODAL 1 ----------
    # Circuito sencillo: 2 nodos (nodo 0 y nodo 1) respecto a tierra.
    # Elementos: R1 entre nodo0 y tierra, R2 entre nodo1 y tierra, R12 entre nodo0 y nodo1.
    # Inyección de corriente conocida en nodo0 e nodo1.
    print("Ejemplo Nodal 1: Red resistiva con fuentes de corriente (2 nodos)")
    R1 = 50.0
    R2 = 100.0
    R12 = 200.0
    Y1g = 1.0 / R1
    Y2g = 1.0 / R2
    Y12 = 1.0 / R12
    # nodos 0 y 1 (no contamos tierra)
    lista_adm = [
        (0, None, Y1g),   # R1 entre 0 y tierra
        (1, None, Y2g),   # R2 entre 1 y tierra
        (0, 1, Y12)       # R12 entre 0 y 1
    ]
    # Inyecciones de corriente (fasores, Vrms no aplicable) — usamos corrientes en A
    I_inj = [
        (0, 0.5 + 0j),   # 0.5 A inyectada en nodo 0
        (1, 0.2 + 0j)    # 0.2 A inyectada en nodo 1
    ]
    G1, I1 = construir_matriz_nodal(2, lista_adm, I_inj)
    Vn1 = resolver_nodal(G1, I1)
    for i, v in enumerate(Vn1):
        m, a = modulo_y_angulo(v)
        print(f"  Nodo {i}: V = {v:.6f} V_rms -> |V| = {m:.6f} V, ángulo = {a:.2f}°")
    print()

    # ---------- EJEMPLO NODAL 2 ----------
    # Circuito RC paralelo con inyección de corriente: encontrar Vn
    print("Ejemplo Nodal 2: Paralelo R || C con corriente inyectada (1 nodo)")
    Rpar = 200.0
    Cpar = 10e-6
    YR = 1.0 / Rpar
    YC = 1j * omega * Cpar
    lista_adm2 = [
        (0, None, YR + YC)  # nodo 0 a tierra con R||C simplificado
    ]
    # Inyección de corriente sinusoidal: I = 0.8 A ∠ -30°
    I_inj2 = [(0, 0.8 * np.exp(1j * grados_a_radianes(-30.0)))]
    G2, I2 = construir_matriz_nodal(1, lista_adm2, I_inj2)
    Vn2 = resolver_nodal(G2, I2)
    v0 = Vn2[0]
    m0, a0 = modulo_y_angulo(v0)
    print(f"  Nodo 0: V = {v0:.6f} V_rms -> |V| = {m0:.6f} V, ángulo = {a0:.2f}°")
    print()

    # ---------- EJEMPLO NODAL 3 ----------
    # Nodal con fuente de tensión conectada al nodo de referencia (modelo simple)
    # Aquí resolvemos un nodo con una fuente de tensión equivalente mediante transformación
    print("Ejemplo Nodal 3: Transformación fuente (tensión a corriente) y solución nodal")
    # Supongamos una fuente de tensión Vs = 120 Vrms ∠ 0 aplicada en serie con Rsource a un nodo con carga Zload a tierra.
    Vs = fasor_desde_vrms(120.0, 0.0)
    Rsource = 10.0
    Zload = 50.0 + 1j * 20.0
    # Transformamos la fuente de tensión Vs en fuente de corriente Norton: I_N = Vs / Rsource, Y_eq = 1/Rsource
    I_norton = Vs / Rsource
    Y_eq = 1.0 / Rsource
    # ahora el nodo tiene inyección I_norton y admitancia total Y_eq + 1/Zload
    Yload = 1.0 / Zload
    lista_adm3 = [(0, None, Y_eq + Yload)]
    I_inj3 = [(0, I_norton)]
    G3, I3 = construir_matriz_nodal(1, lista_adm3, I_inj3)
    Vn3 = resolver_nodal(G3, I3)
    v3 = Vn3[0]
    m3, a3 = modulo_y_angulo(v3)
    print(f"  Nodo (carga): V = {v3:.6f} V_rms -> |V| = {m3:.6f} V, ángulo = {a3:.2f}°")
    print()

    # ---------- EJEMPLO MALLAS 1 ----------
    # Circuito de dos mallas con fuentes de tensión: calcular corrientes de malla
    print("Ejemplo Mallas 1: Dos mallas con fuentes (activar método de mallas)")
    # Definimos impedancias en cada rama (ohm) a 60 Hz
    R_a = 10.0
    L_a = 20e-3
    R_b = 15.0
    L_b = 10e-3
    Z1 = impedancia_R(R_a) + impedancia_L(omega, L_a)    # rama malla 1
    Z2 = impedancia_R(R_b) + impedancia_L(omega, L_b)    # rama malla 2
    Z12 = 5.0  # impedancia compartida entre mallas
    # Matriz de impedancias de mallas (2x2):
    # [Z1 + Z12, -Z12
    #  -Z12,     Z2 + Z12]
    Zm = [
        [Z1 + Z12, -Z12],
        [-Z12, Z2 + Z12]
    ]
    # Fuentes: supongamos V1 = 120 Vrms ∠0 en malla 1, V2 = 60 Vrms ∠ -30° en malla 2 (signo convención)
    V1 = fasor_desde_vrms(120.0, 0.0)
    V2 = fasor_desde_vrms(60.0, -30.0)
    Zm_mat, Vvec = construir_matriz_mallas(Zm, [V1, V2])
    Im = resolver_mallas(Zm_mat, Vvec)
    for i, cur in enumerate(Im):
        mcur, acur = modulo_y_angulo(cur)
        print(f"  Corriente de malla I{i+1} = {cur:.6f} A -> |I| = {mcur:.6f} A, ángulo = {acur:.2f}°")
    print()

    # ---------- EJEMPLO MALLAS 2 ----------
    # Malla con fuente dependiente (ejemplo simplificado): resolvemos con matriz 3x3
    print("Ejemplo Mallas 2: Tres mallas con varias impedancias (ejemplo numérico)")
    Z11 = 10.0 + 1j * 5.0
    Z12 = -2.0  # acoplamiento (negativo)
    Z13 = -1.0
    Z22 = 8.0 + 1j * 3.0
    Z23 = -1.5
    Z33 = 12.0 + 1j * 4.0
    Zm2 = [
        [Z11, Z12, Z13],
        [Z12, Z22, Z23],
        [Z13, Z23, Z33]
    ]
    # Fuentes de malla (Vrms)
    V_mallas = [100.0 * np.exp(1j * grados_a_radianes(0.0)),
                50.0 * np.exp(1j * grados_a_radianes(-20.0)),
                0.0]
    Zm2_mat, V_m2 = construir_matriz_mallas(Zm2, V_mallas)
    Im2 = resolver_mallas(Zm2_mat, V_m2)
    for i, cur in enumerate(Im2):
        mcur, acur = modulo_y_angulo(cur)
        print(f"  Corriente de malla I{i+1} = {cur:.6f} A -> |I| = {mcur:.6f} A, ángulo = {acur:.2f}°")
    print()

if __name__ == "__main__":
    resolver_ejemplos()


# ***Potencia en Corriente Alterna***

# Potencia en corriente alterna

La potencia compleja se define como $ \tilde{S} = \tilde{V}_{rms} \, \tilde{I}_{rms}^{*} = P + jQ $,   (donde * es conjugado).

donde

Potencia activa: $ P = \Re{\tilde{S}} $

Potencia reactiva: $ Q = \Im{\tilde{S}} $

Potencia aparente: $ |\tilde{S}| = V_{rms} I_{rms} $

Factor de potencia: $ pf = \dfrac{P}{|\tilde{S}|} = \cos \phi $

---

Paso a paso:

1. Representar $ \tilde{V}{rms} = V{rms} \angle \theta_V $, $ \tilde{I}{rms} = I{rms} \angle \theta_I $.

2. Calcular $ \tilde{S} = \tilde{V}{rms} , \tilde{I}{rms}^* $.

3. Hallar $ P = \Re(\tilde{S}) $, $ Q = \Im(\tilde{S}) $, $ |\tilde{S}| = \sqrt{P^2 + Q^2} $.

4. Factor de potencia $ pf = \cos(\theta_V - \theta_I) $.

5. Para corrección: $ Q_{nuevo} = P \tan(\arccos(pf_{objetivo})) $, $ Q_c = Q - Q_{nuevo} $.

---


In [ ]:
import numpy as np

# --- Funciones auxiliares ---
def grados_a_radianes(g):
    return g * np.pi / 180.0

def radianes_a_grados(r):
    return r * 180.0 / np.pi

def modulo_y_angulo(z):
    return np.abs(z), radianes_a_grados(np.angle(z))

def fasor(Vrms, angulo_grados=0.0):
    """Crea un fasor complejo a partir de un valor RMS y su ángulo (en grados)."""
    return Vrms * np.exp(1j * grados_a_radianes(angulo_grados))

def impedancia_R(R):
    return complex(R)

def impedancia_L(omega, L):
    return 1j * omega * L

def impedancia_C(omega, C):
    return 1.0 / (1j * omega * C)

# --- Cálculo de potencias ---
def calcular_potencias(V, I):
    """
    Calcula las potencias a partir de los fasores V e I.
    Retorna (P, Q, S, pf)
    """
    S = V * np.conj(I)
    P = np.real(S)
    Q = np.imag(S)
    S_abs = np.abs(S)
    pf = P / S_abs if S_abs != 0 else 0
    return P, Q, S_abs, pf

# --- Ejemplos ---
def resolver_ejemplos():
    print("=== POTENCIA EN CORRIENTE ALTERNA ===\n")

    f = 60.0
    omega = 2.0 * np.pi * f

    # ----------------------------------------------------------
    print("Ejemplo 1: Carga puramente resistiva (R = 20 Ω, 120 Vrms)")
    Vrms = 120.0
    R = 20.0
    Z = impedancia_R(R)
    I = fasor(Vrms, 0.0) / Z
    P, Q, S, pf = calcular_potencias(fasor(Vrms, 0.0), I)
    print(f"Z = {Z:.2f} Ω")
    print(f"I = {I:.4f} A")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.2f} (unitario)\n")

    # ----------------------------------------------------------
    print("Ejemplo 2: Carga RL (R = 30 Ω, L = 100 mH, 120 Vrms)")
    R = 30.0
    L = 100e-3
    Z = R + impedancia_L(omega, L)
    V = fasor(120.0, 0.0)
    I = V / Z
    P, Q, S, pf = calcular_potencias(V, I)
    print(f"Z = {Z:.3f} Ω")
    print(f"I = {I:.4f} A")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f} (atrasado)\n")

    # ----------------------------------------------------------
    print("Ejemplo 3: Carga RC (R = 50 Ω, C = 100 µF, 120 Vrms)")
    R = 50.0
    C = 100e-6
    Z = R + impedancia_C(omega, C)
    V = fasor(120.0, 0.0)
    I = V / Z
    P, Q, S, pf = calcular_potencias(V, I)
    print(f"Z = {Z:.3f} Ω")
    print(f"I = {I:.4f} A")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f} (adelantado)\n")

    # ----------------------------------------------------------
    print("Ejemplo 4: Método corto (Vrms = 40, Z = 11.88 + j4.47 Ω)")
    Vrms = 40.0
    Vm = Vrms * np.sqrt(2)
    Z = 11.88 + 1j * 4.47
    Im = Vm / np.abs(Z)
    ang_Z = np.angle(Z)
    pf = np.cos(ang_Z)
    P = (Vm * Im / 2) * pf
    Q = (Vm * Im / 2) * np.sin(ang_Z)
    S = (Vm * Im) / 2
    print(f"Z = {Z:.3f} Ω")
    print(f"Vm = {Vm:.3f} V, Im = {Im:.3f} A")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f}\n")

    # ----------------------------------------------------------
    print("Ejemplo 5: Corrección del factor de potencia (RL con capacitor en paralelo)")
    R = 30.0
    L = 100e-3
    C = 200e-6
    V = fasor(120.0, 0.0)
    Z_RL = R + impedancia_L(omega, L)
    I_RL = V / Z_RL
    P, Q, S, pf = calcular_potencias(V, I_RL)
    print("→ Antes de la corrección:")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f} (atrasado)\n")

    # Corriente del capacitor (en paralelo)
    I_C = V / impedancia_C(omega, C)
    I_total = I_RL + I_C
    P_corr, Q_corr, S_corr, pf_corr = calcular_potencias(V, I_total)
    print("→ Después de agregar el capacitor:")
    print(f"P = {P_corr:.2f} W")
    print(f"Q = {Q_corr:.2f} var")
    print(f"S = {S_corr:.2f} VA")
    print(f"pf = {pf_corr:.3f} (corregido)\n")

if __name__ == "__main__":
    resolver_ejemplos()

Ejemplo Potencia 1:
S = 1039.230+600.000j VA, P = 1039.230 W, Q = 600.000 var, |S|=1200.000 VA, pf=0.8660
Ejemplo Potencia 2 (método corto):
P = 324.750 W
Ejemplo Potencia 3 (corrección PF):
Para pf_target=0.95, Q_new=341.579 var, Qc a instalar = 258.421 var (positivo => debe suministrar esa var)



# ***Redes trifásicas***

Sistemas balanceados:

* En estrella (Y): $ V_{fase} = \dfrac{V_{línea}}{\sqrt{3}} $, $ I_{línea} = I_{fase} $.

* En delta (Δ): $ V_{fase} = V_{línea} $, $ I_{línea} = \sqrt{3} , I_{fase} $.

Potencias totales:

 $$ P_{tot} = \sqrt{3} \, V_{línea} \, I_{línea} \cos \phi $$

 $$ Q_{tot} = \sqrt{3} \, V_{línea} \, I_{línea} \sin \phi $$

 $$ S_{tot} = \sqrt{3} \, V_{línea} \, I_{línea} $$

---

Paso a paso (Y balanceada):

1. Calcular $ V_{fase} = V_{línea} / \sqrt{3} $.

2. Calcular $ I_{fase} = V_{fase} / Z_{fase} $.

3. Calcular $ S_{fase} = V_{fase} I_{fase}^* $.

4. Hallar potencias totales: $ P_{tot} = 3 \Re(S_{fase}) $, $ Q_{tot} = 3 \Im(S_{fase}) $.
---

In [1]:
import numpy as np

# --- Funciones auxiliares ---
def grados_a_radianes(g):
    return g * np.pi / 180.0

def radianes_a_grados(r):
    return r * 180.0 / np.pi

def modulo_y_angulo(z):
    return np.abs(z), radianes_a_grados(np.angle(z))

def fasor(Vrms, angulo_grados=0.0):
    return Vrms * np.exp(1j * grados_a_radianes(angulo_grados))

def impedancia_R(R):
    return complex(R)

def impedancia_L(omega, L):
    return 1j * omega * L

def impedancia_C(omega, C):
    return 1.0 / (1j * omega * C)

# --- Cálculo de potencias trifásicas ---
def calcular_potencias_trifasicas(V_linea, I_linea, angulo_phi):
    P = np.sqrt(3) * V_linea * I_linea * np.cos(angulo_phi)
    Q = np.sqrt(3) * V_linea * I_linea * np.sin(angulo_phi)
    S = np.sqrt(3) * V_linea * I_linea
    pf = np.cos(angulo_phi)
    return P, Q, S, pf

# --- Ejemplos ---
def resolver_ejemplos():
    print("=== REDES TRIFÁSICAS ===\n")

    f = 60.0
    omega = 2 * np.pi * f

    # ----------------------------------------------------------
    print("Ejemplo 1: Sistema trifásico Y balanceado (R-L por fase)")
    V_linea = 208.0
    V_fase = V_linea / np.sqrt(3)
    R = 20.0
    L = 50e-3
    Z = R + impedancia_L(omega, L)
    I_fase = V_fase / Z
    I_linea = np.abs(I_fase)
    angulo_phi = np.angle(Z)
    P, Q, S, pf = calcular_potencias_trifasicas(V_linea, I_linea, angulo_phi)
    print(f"Z_fase = {Z:.3f} Ω")
    print(f"I_fase = {I_fase:.4f} A")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f} (atrasado)\n")

    # ----------------------------------------------------------
    print("Ejemplo 2: Sistema trifásico Δ balanceado (R-C por fase)")
    V_linea = 220.0
    V_fase = V_linea  # En delta
    R = 40.0
    C = 100e-6
    Z = R + impedancia_C(omega, C)
    I_fase = V_fase / Z
    I_linea = np.sqrt(3) * np.abs(I_fase)
    angulo_phi = np.angle(Z)
    P, Q, S, pf = calcular_potencias_trifasicas(V_linea, I_linea, angulo_phi)
    print(f"Z_fase = {Z:.3f} Ω")
    print(f"I_linea = {I_linea:.4f} A")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f} (adelantado)\n")

    # ----------------------------------------------------------
    print("Ejemplo 3: Sistema Y desbalanceado (3 cargas diferentes)")
    V_linea = 208.0
    V_fase = V_linea / np.sqrt(3)
    Z_A = 20 + 1j * 10
    Z_B = 25 - 1j * 15
    Z_C = 30 + 1j * 5
    V_A = fasor(V_fase, 0)
    V_B = fasor(V_fase, -120)
    V_C = fasor(V_fase, 120)
    I_A = V_A / Z_A
    I_B = V_B / Z_B
    I_C = V_C / Z_C
    S_A = V_A * np.conj(I_A)
    S_B = V_B * np.conj(I_B)
    S_C = V_C * np.conj(I_C)
    S_total = S_A + S_B + S_C
    P = np.real(S_total)
    Q = np.imag(S_total)
    S = np.abs(S_total)
    pf = P / S
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f}\n")

    # ----------------------------------------------------------
    print("Ejemplo 4: Sistema Δ balanceado (carga RL)")
    V_linea = 220.0
    V_fase = V_linea
    R = 50.0
    L = 80e-3
    Z = R + impedancia_L(omega, L)
    I_fase = V_fase / Z
    I_linea = np.sqrt(3) * np.abs(I_fase)
    angulo_phi = np.angle(Z)
    P, Q, S, pf = calcular_potencias_trifasicas(V_linea, I_linea, angulo_phi)
    print(f"Z_fase = {Z:.3f} Ω")
    print(f"I_linea = {I_linea:.4f} A")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f} (atrasado)\n")

    # ----------------------------------------------------------
    print("Ejemplo 5: Corrección del factor de potencia trifásico (Y balanceado)")
    V_linea = 208.0
    V_fase = V_linea / np.sqrt(3)
    R = 30.0
    L = 100e-3
    Z = R + impedancia_L(omega, L)
    I_fase = V_fase / Z
    I_linea = np.abs(I_fase)
    ang_phi_inicial = np.angle(Z)
    P, Q, S, pf = calcular_potencias_trifasicas(V_linea, I_linea, ang_phi_inicial)
    print("→ Antes de corrección:")
    print(f"P = {P:.2f} W")
    print(f"Q = {Q:.2f} var")
    print(f"S = {S:.2f} VA")
    print(f"pf = {pf:.3f}\n")

    # Corrección con capacitor (buscamos Qc que reduzca Q a 0)
    Qc_por_fase = Q / 3.0  # Q por fase
    V_rms_fase = V_fase
    Xc = (V_rms_fase ** 2) / Qc_por_fase
    C = 1 / (omega * Xc)
    Z_corr = 1.0 / (1.0 / Z + 1j * omega * C)
    I_fase_corr = V_fase / Z_corr
    I_linea_corr = np.abs(I_fase_corr)
    ang_phi_corr = np.angle(Z_corr)
    P_corr, Q_corr, S_corr, pf_corr = calcular_potencias_trifasicas(V_linea, I_linea_corr, ang_phi_corr)
    print("→ Después de corrección:")
    print(f"P = {P_corr:.2f} W")
    print(f"Q = {Q_corr:.2f} var")
    print(f"S = {S_corr:.2f} VA")
    print(f"pf = {pf_corr:.3f} (corregido)\n")

if __name__ == "__main__":
    resolver_ejemplos()


=== REDES TRIFÁSICAS ===

Ejemplo 1: Sistema trifásico Y balanceado (R-L por fase)
Z_fase = 20.000+18.850j Ω
I_fase = 3.1799-2.9970j A
P = 1145.60 W
Q = 1079.70 var
S = 1574.22 VA
pf = 0.728 (atrasado)

Ejemplo 2: Sistema trifásico Δ balanceado (R-C por fase)
Z_fase = 40.000-26.526j Ω
I_linea = 7.9392 A
P = 2521.25 W
Q = -1671.96 var
S = 3025.25 VA
pf = 0.833 (adelantado)

Ejemplo 3: Sistema Y desbalanceado (3 cargas diferentes)
P = 1468.73 W
Q = 111.89 var
S = 1472.98 VA
pf = 0.997

Ejemplo 4: Sistema Δ balanceado (carga RL)
Z_fase = 50.000+30.159j Ω
I_linea = 6.5258 A
P = 2129.29 W
Q = 1284.36 var
S = 2486.66 VA
pf = 0.856 (atrasado)

Ejemplo 5: Corrección del factor de potencia trifásico (Y balanceado)
→ Antes de corrección:
P = 559.15 W
Q = 702.65 var
S = 897.98 VA
pf = 0.623

→ Después de corrección:
P = 559.15 W
Q = 0.00 var
S = 559.15 VA
pf = 1.000 (corregido)



# ***Redes acopladas magnéticamente***

Ecuaciones fasoriales:

$$
\begin{cases}
\tilde{V}_1 = j \omega L_1 \tilde{I}_1 + j \omega M \tilde{I}_2 \\\\
\tilde{V}_2 = j \omega M \tilde{I}_1 + j \omega L_2 \tilde{I}_2
\end{cases}
$$

Coeficiente de acoplamiento: $ k = \dfrac{M}{\sqrt{L_1 L_2}} $

Energía almacenada: $ W = \tfrac{1}{2} L_1 I_1^2 + \tfrac{1}{2} L_2 I_2^2 + M I_1 I_2 $

---
Paso a paso:

1. Calcular $ j \omega L_1 $, $ j \omega L_2 $, $ j \omega M $.

2. Formar la matriz de impedancias:

$$
Z =
\begin{bmatrix}
R_1 + j \omega L_1 & j \omega M \\\\
j \omega M & R_2 + j \omega L_2
\end{bmatrix}
$$

3. Resolver $ \mathbf{I} = Z^{-1} \mathbf{V} $.

4. Obtener $ I_1, I_2 $ y calcular energía si es necesario.
---

In [ ]:
import numpy as np

# --- Utilidades ---
def grados_a_radianes(g):
    return g * np.pi / 180.0

def radianes_a_grados(r):
    return r * 180.0 / np.pi

def modulo_y_angulo(z):
    return np.abs(z), radianes_a_grados(np.angle(z))

def fasor(Vrms, angulo_grados=0.0):
    """Crea un fasor complejo a partir de Vrms y ángulo en grados."""
    return Vrms * np.exp(1j * grados_a_radianes(angulo_grados))

# --- Construcción de la matriz de impedancias acopladas ---
def matriz_impedancias_acopladas(omega, L1, L2, M, R1=0.0, R2=0.0):
    """
    Construye la matriz Z (2x2) para dos inductancias acopladas con resistencias en serie.
    Z11 = R1 + j ω L1
    Z22 = R2 + j ω L2
    Z12 = Z21 = j ω M
    """
    Z11 = R1 + 1j * omega * L1
    Z22 = R2 + 1j * omega * L2
    Z12 = 1j * omega * M
    Z = np.array([[Z11, Z12],
                  [Z12, Z22]], dtype=complex)
    return Z

# --- Resolver circuito acoplado ---
def resolver_circuito_acoplado(Z, V1, V2):
    """
    Resuelve Z · I = V para obtener I1, I2 (fasores).
    V1, V2 son fasores (Vrms).
    Retorna (I1, I2)
    """
    V = np.array([V1, V2], dtype=complex)
    I = np.linalg.solve(Z, V)
    return I[0], I[1]

# --- Cálculo del coeficiente de acoplamiento y energía ---
def coeficiente_acoplamiento(M, L1, L2):
    """k = M / sqrt(L1 * L2)"""
    return M / np.sqrt(L1 * L2)

def energia_almacenada(L1, L2, M, I1, I2):
    """
    Calcula la energía aproximada almacenada:
    W = 0.5*L1*|I1|^2 + 0.5*L2*|I2|^2 + M * Re(I1 * conj(I2))
    (se usa la parte real del producto complejo para la contribución mixta)
    """
    term1 = 0.5 * L1 * (np.abs(I1) ** 2)
    term2 = 0.5 * L2 * (np.abs(I2) ** 2)
    term3 = M * np.real(I1 * np.conjugate(I2))
    W = term1 + term2 + term3
    return W

# --- Ejemplos (5) ---
def resolver_ejemplos():
    print("=== REDES ACOPLADAS MAGNÉTICAMENTE — Ejemplos ===\n")

    f = 60.0
    omega = 2.0 * np.pi * f

    # ----------------- Ejemplo 1 -----------------
    print("Ejemplo 1: Dos inductores acoplados, V2 = 0 (segunda bobina en circuito abierto salvo por RL)")
    L1 = 50e-3    # 50 mH
    L2 = 20e-3    # 20 mH
    M  = 8e-3     # 8 mH mutua
    R1 = 5.0      # resistencia serie 1 (ohm)
    R2 = 2.0      # resistencia serie 2 (ohm)
    V1 = fasor(120.0, 0.0)  # fuente en primaria 120 Vrms ∠0
    V2 = 0.0 + 0.0j          # secundario sin fuente (puede estar conectado a carga en otros ejemplos)
    Z = matriz_impedancias_acopladas(omega, L1, L2, M, R1, R2)
    I1, I2 = resolver_circuito_acoplado(Z, V1, V2)
    k = coeficiente_acoplamiento(M, L1, L2)
    W = energia_almacenada(L1, L2, M, I1, I2)
    m1,a1 = modulo_y_angulo(I1)
    m2,a2 = modulo_y_angulo(I2)
    print(f"I1 = {I1:.6f} A -> |I1| = {m1:.6f} A, ángulo = {a1:.2f}°")
    print(f"I2 = {I2:.6f} A -> |I2| = {m2:.6f} A, ángulo = {a2:.2f}°")
    print(f"k = {k:.4f}")
    print(f"Energía aproximada W = {W:.6f} J")
    print()

    # ----------------- Ejemplo 2 -----------------
    print("Ejemplo 2: Ambas bobinas alimentadas (V1 y V2 no nulas), calcular corrientes")
    L1 = 30e-3
    L2 = 30e-3
    M  = 10e-3
    R1 = 3.0
    R2 = 3.0
    V1 = fasor(100.0, 0.0)
    V2 = fasor(50.0, -30.0)
    Z = matriz_impedancias_acopladas(omega, L1, L2, M, R1, R2)
    I1, I2 = resolver_circuito_acoplado(Z, V1, V2)
    k = coeficiente_acoplamiento(M, L1, L2)
    W = energia_almacenada(L1, L2, M, I1, I2)
    m1,a1 = modulo_y_angulo(I1)
    m2,a2 = modulo_y_angulo(I2)
    print(f"I1 = {I1:.6f} A -> |I1| = {m1:.6f} A, ángulo = {a1:.2f}°")
    print(f"I2 = {I2:.6f} A -> |I2| = {m2:.6f} A, ángulo = {a2:.2f}°")
    print(f"k = {k:.4f}")
    print(f"Energía aproximada W = {W:.6f} J")
    print()

    # ----------------- Ejemplo 3 -----------------
    print("Ejemplo 3: Calcular M a partir de k dado y L1,L2 (verificación) y resolver circuito con carga en secundario")
    L1 = 20e-3
    L2 = 5e-3
    k_dado = 0.7
    M = k_dado * np.sqrt(L1 * L2)   # M calculada desde k
    R1 = 2.0
    R2 = 10.0   # supongamos carga mayor en secundaria
    V1 = fasor(120.0, 0.0)
    V2 = 0.0
    Z = matriz_impedancias_acopladas(omega, L1, L2, M, R1, R2)
    I1, I2 = resolver_circuito_acoplado(Z, V1, V2)
    k_calc = coeficiente_acoplamiento(M, L1, L2)
    W = energia_almacenada(L1, L2, M, I1, I2)
    m1,a1 = modulo_y_angulo(I1)
    m2,a2 = modulo_y_angulo(I2)
    print(f"M calculada = {M:.6e} H (desde k = {k_dado})")
    print(f"I1 = {I1:.6f} A -> |I1| = {m1:.6f} A, ángulo = {a1:.2f}°")
    print(f"I2 = {I2:.6f} A -> |I2| = {m2:.6f} A, ángulo = {a2:.2f}°")
    print(f"k (verificación) = {k_calc:.4f}")
    print(f"Energía aproximada W = {W:.6f} J")
    print()

    # ----------------- Ejemplo 4 -----------------
    print("Ejemplo 4: Matriz Z numérica y resolución para V1=10∠0 V, V2=5∠0 V (alto acoplamiento pequeño)")
    L1 = 0.1
    L2 = 0.05
    M  = 0.02
    R1 = 1.0
    R2 = 2.0
    V1 = fasor(10.0, 0.0)
    V2 = fasor(5.0, 0.0)
    Z = matriz_impedancias_acopladas(omega, L1, L2, M, R1, R2)
    I1, I2 = resolver_circuito_acoplado(Z, V1, V2)
    k = coeficiente_acoplamiento(M, L1, L2)
    W = energia_almacenada(L1, L2, M, I1, I2)
    m1,a1 = modulo_y_angulo(I1)
    m2,a2 = modulo_y_angulo(I2)
    print("Matriz Z =")
    print(f"  Z11 = {Z[0,0]:.6f}, Z12 = {Z[0,1]:.6f}")
    print(f"  Z21 = {Z[1,0]:.6f}, Z22 = {Z[1,1]:.6f}")
    print(f"I1 = {I1:.6f} A -> |I1| = {m1:.6f} A, ángulo = {a1:.2f}°")
    print(f"I2 = {I2:.6f} A -> |I2| = {m2:.6f} A, ángulo = {a2:.2f}°")
    print(f"k = {k:.4f}")
    print(f"Energía aproximada W = {W:.6f} J")
    print()

    # ----------------- Ejemplo 5 -----------------
    print("Ejemplo 5: Energía almacenada en función de corrientes impuestas (verificación de signo cruzado)")
    L1 = 0.02
    L2 = 0.02
    M  = 0.015
    # Supongamos corrientes dadas (fasores, ejemplo pedagógico)
    I1 = 2.0 * np.exp(1j * grados_a_radianes(20.0))   # 2 A ∠20°
    I2 = 1.5 * np.exp(1j * grados_a_radianes(-10.0))  # 1.5 A ∠-10°
    W = energia_almacenada(L1, L2, M, I1, I2)
    m1,a1 = modulo_y_angulo(I1)
    m2,a2 = modulo_y_angulo(I2)
    print(f"I1 impuesta = {I1:.6f} A -> |I1| = {m1:.6f} A, ángulo = {a1:.2f}°")
    print(f"I2 impuesta = {I2:.6f} A -> |I2| = {m2:.6f} A, ángulo = {a2:.2f}°")
    print(f"L1 = {L1} H, L2 = {L2} H, M = {M} H")
    print(f"Energía almacenada W = {W:.6f} J")
    print()

if __name__ == "__main__":
    resolver_ejemplos()


# ***Transformador ideal***

Relaciones fundamentales:

$$
\frac{V_1}{V_2} = \frac{N_1}{N_2} = a
$$

$$
\frac{I_2}{I_1} = a
$$

$$
Z'_L = a^2 Z_L
$$

$$
S_1 = S_2
$$

Impedancia reflejada: $ Z'_L = a^2 Z_L $

Potencia conservada: $ S_1 = S_2 $

---
Paso a paso:

1. Calcular razón de espiras $ a = N_1 / N_2 $.

2. Reflejar la carga $ Z'_L = a^2 Z_L $.

3. Calcular impedancia total $ Z_{tot} = R_s + Z'_L $.

4. Corriente primaria $ I_1 = V_s / Z_{tot} $.

5. Corriente secundaria $ I_2 = a , I_1 $.

6. Tensión secundaria $ V_2 = V_1 / a $.

7. Verificar $ P_1 \approx P_2 $.
---

In [ ]:
import numpy as np

# --- Funciones auxiliares ---
def grados_a_radianes(g):
    return g * np.pi / 180.0

def radianes_a_grados(r):
    return r * 180.0 / np.pi

def modulo_y_angulo(z):
    return np.abs(z), radianes_a_grados(np.angle(z))

def fasor(Vrms, angulo_grados=0.0):
    """Crea un fasor complejo a partir de Vrms y ángulo en grados."""
    return Vrms * np.exp(1j * grados_a_radianes(angulo_grados))

def calcular_potencia(V, I):
    """Devuelve P, Q, S y factor de potencia a partir de fasores."""
    S = V * np.conj(I)
    P = np.real(S)
    Q = np.imag(S)
    Sabs = np.abs(S)
    pf = P / Sabs if Sabs != 0 else 0
    return P, Q, Sabs, pf

# --- Funciones del transformador ideal ---
def relacion_espiras(N1, N2):
    """Calcula la relación de transformación a = N1 / N2."""
    return N1 / N2

def reflejar_impedancia(Z_L, a):
    """Refleja la impedancia de carga al primario: Z' = a² * Z_L"""
    return (a ** 2) * Z_L

# --- Ejemplos ---
def resolver_ejemplos():
    print("=== TRANSFORMADOR IDEAL ===\n")

    f = 60.0
    omega = 2.0 * np.pi * f

    # ----------------------------------------------------------
    print("Ejemplo 1: Transformador 1:5 (reductor) — relación de tensiones y corrientes")
    N1 = 1000
    N2 = 5000
    a = relacion_espiras(N1, N2)
    V1 = 120.0
    V2 = V1 / a
    I2 = 10.0
    I1 = I2 / a
    print(f"a = N1/N2 = {a:.2f}")
    print(f"V1 = {V1:.2f} V, V2 = {V2:.2f} V")
    print(f"I1 = {I1:.2f} A, I2 = {I2:.2f} A")
    print()

    # ----------------------------------------------------------
    print("Ejemplo 2: Transformador 10:1 (elevador) con carga RL en el secundario")
    N1 = 100
    N2 = 10
    a = relacion_espiras(N1, N2)
    V1 = fasor(240.0, 0.0)
    ZL = 20 + 1j * 10
    V2 = V1 / a
    I2 = V2 / ZL
    I1 = I2 / a
    P2, Q2, S2, pf2 = calcular_potencia(V2, I2)
    P1, Q1, S1, pf1 = calcular_potencia(V1, I1)
    print(f"a = {a:.2f}")
    print(f"Z_L = {ZL:.3f} Ω")
    print(f"V1 = {V1:.3f} V, V2 = {V2:.3f} V")
    print(f"I1 = {I1:.4f} A, I2 = {I2:.4f} A")
    print(f"P1 = {P1:.2f} W, P2 = {P2:.2f} W (≈ iguales)")
    print(f"pf = {pf1:.3f}")
    print()

    # ----------------------------------------------------------
    print("Ejemplo 3: Impedancia reflejada — determinar corriente en el primario")
    N1 = 500
    N2 = 100
    a = relacion_espiras(N1, N2)
    ZL = 8 + 1j * 6
    Z_ref = reflejar_impedancia(ZL, a)
    V1 = fasor(120.0, 0.0)
    I1 = V1 / Z_ref
    V2 = V1 / a
    I2 = I1 * a
    P1, Q1, S1, pf1 = calcular_potencia(V1, I1)
    print(f"a = {a:.2f}")
    print(f"ZL = {ZL:.3f} Ω -> Z_ref = {Z_ref:.3f} Ω")
    print(f"I1 = {I1:.4f} A, I2 = {I2:.4f} A")
    print(f"P = {P1:.2f} W")
    print(f"Q = {Q1:.2f} var")
    print(f"S = {S1:.2f} VA")
    print(f"pf = {pf1:.3f}")
    print()

    # ----------------------------------------------------------
    print("Ejemplo 4: Transformador ideal con carga RLC — potencia en ambos lados")
    N1 = 200
    N2 = 1000
    a = relacion_espiras(N1, N2)
    R = 50.0
    L = 50e-3
    C = 100e-6
    ZL = R + 1j * omega * L + 1 / (1j * omega * C)
    V1 = fasor(120.0, 0.0)
    V2 = V1 / a
    I2 = V2 / ZL
    I1 = I2 / a
    P2, Q2, S2, pf2 = calcular_potencia(V2, I2)
    P1, Q1, S1, pf1 = calcular_potencia(V1, I1)
    print(f"a = {a:.2f}")
    print(f"ZL = {ZL:.3f} Ω")
    print(f"I1 = {I1:.4f} A, I2 = {I2:.4f} A")
    print(f"P1 = {P1:.2f} W, P2 = {P2:.2f} W (≈ iguales)")
    print(f"pf = {pf1:.3f}")
    print()

    # ----------------------------------------------------------
    print("Ejemplo 5: Comparación con circuito acoplado equivalente (referencia de M)")
    L1 = 40e-3
    L2 = 10e-3
    M = 20e-3
    k = M / np.sqrt(L1 * L2)
    N1 = 400
    N2 = 200
    a = relacion_espiras(N1, N2)
    print(f"Coeficiente de acoplamiento (k) ≈ {k:.3f}, relación de espiras a = {a:.2f}")
    ZL = 25 + 1j * 10
    Z_ref = reflejar_impedancia(ZL, a)
    V1 = fasor(240.0, 0.0)
    I1 = V1 / Z_ref
    V2 = V1 / a
    I2 = I1 * a
    P1, Q1, S1, pf1 = calcular_potencia(V1, I1)
    P2, Q2, S2, pf2 = calcular_potencia(V2, I2)
    print(f"Z_ref = {Z_ref:.3f} Ω")
    print(f"I1 = {I1:.4f} A, I2 = {I2:.4f} A")
    print(f"P1 = {P1:.2f} W, P2 = {P2:.2f} W (≈ iguales)")
    print(f"pf = {pf1:.3f}")
    print()

if __name__ == "__main__":
    resolver_ejemplos()

# ***Respuesta en Frecuencia***

La **respuesta en frecuencia** describe cómo un circuito lineal (como un filtro) modifica la **magnitud** y la **fase** de una señal al variar la frecuencia.  
Se analiza mediante la **función de transferencia**:

$$
H(j\omega) = \frac{V_{out}(j\omega)}{V_{in}(j\omega)}
$$

La magnitud $|H(j\omega)|$ determina la **ganancia** del circuito, y el argumento $\angle H(j\omega)$ determina el **desfase**.  
La ganancia en decibelios se expresa como:

$$
G_{dB} = 20 \log_{10} |H(j\omega)|
$$


### Filtros pasivos
- RC pasa bajos: $ H(j\omega) = \dfrac{1}{1 + j\omega R C} $
- RC pasa altos: $ H(j\omega) = \dfrac{j\omega R C}{1 + j\omega R C} $
- RL pasa bajos: $ H(j\omega) = \dfrac{R}{R + j\omega L} $
- RL pasa altos: $ H(j\omega) = \dfrac{j\omega L}{R + j\omega L} $
- RLC pasa banda: $ H(j\omega) = \dfrac{j\omega L / R}{1 - \omega^2 L C + j\omega L / R} $

La frecuencia de corte o resonancia se calcula como:
$$
\omega_c = \frac{1}{R C} \quad \text{o} \quad \omega_0 = \frac{1}{\sqrt{L C}}
$$


### Filtros activos (Sallen–Key)
El amplificador operacional permite obtener mayores ganancias y pendientes más pronunciadas.  
Ejemplo de filtro Sallen–Key pasa bajos:

$$
H(j\omega) = \frac{A_{ol}}{1 + j\omega (R_1 C_1 + R_2 C_1 + R_2 C_2 (1 - A_{ol})) + (j\omega)^2 R_1 R_2 C_1 C_2}
$$

Y pasa altos:

$$
H(j\omega) = \frac{A_{ol} (j\omega)^2 R_1 R_2 C_1 C_2}{1 + j\omega (R_1 C_1 + R_2 C_1 + R_2 C_2 (1 - A_{ol})) + (j\omega)^2 R_1 R_2 C_1 C_2}
$$

---

### Paso a paso (cómo resolver a mano un ejercicio)
1. Escribir la función de transferencia $H(j\omega)$ del circuito.  
2. Sustituir los valores de R, L, C y obtener la expresión compleja.  
3. Calcular $|H(j\omega)|$ y $\angle H(j\omega)$ para distintas frecuencias.  
4. Determinar la frecuencia de corte ($\omega_c$ o $f_c$).  
5. Representar $20\log_{10}|H|$ (ganancia en dB) y la fase en grados en función de la frecuencia (diagrama de Bode).  
6. Analizar regiones: baja, de transición y alta frecuencia.

---

### Observaciones
- En filtros **pasa bajos**, la ganancia es alta a bajas frecuencias y cae a –20 dB/dec a partir de $f_c$.  
- En **pasa altos**, la ganancia es baja a bajas frecuencias y sube tras $f_c$.  
- En **pasa banda**, hay una resonancia alrededor de $f_0$.  
- En filtros **activos**, la ganancia puede ser mayor que 1 (0 dB).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Utilidades matemáticas
def db(x):
    return 20 * np.log10(np.abs(x))

def phase_deg(x):
    return np.angle(x, deg=True)

# --- Filtros Pasivos ---

def H_RC_lowpass(R, C, omega):
    # H(jw) = 1 / (1 + jωRC)
    return 1 / (1 + 1j * omega * R * C)

def H_RC_highpass(R, C, omega):
    # H(jw) = jωRC / (1 + jωRC)
    return (1j * omega * R * C) / (1 + 1j * omega * R * C)

def H_RL_lowpass(R, L, omega):
    # H(jw) = R / (R + jωL)
    return R / (R + 1j * omega * L)

def H_RL_highpass(R, L, omega):
    # H(jw) = jωL / (R + jωL)
    return (1j * omega * L) / (R + 1j * omega * L)

def H_RLC_bandpass(R, L, C, omega):
    # H(jw) = (jωL/R) / (1 - ω^2LC + jωL/R)
    num = (1j * omega * L / R)
    den = (1 - (omega ** 2) * L * C + 1j * omega * L / R)
    return num / den

# --- Filtros Activos (Sallen–Key) ---

def H_SallenKey_lowpass(R1, R2, C1, C2, A_ol, omega):
    # Filtro pasa bajos activo (Sallen-Key)
    s = 1j * omega
    H = (A_ol) / (1 + s * (R1 * C1 + R2 * C1 + R2 * C2 * (1 - A_ol)) + (s**2) * R1 * R2 * C1 * C2)
    return H

def H_SallenKey_highpass(R1, R2, C1, C2, A_ol, omega):
    # Filtro pasa altos activo (Sallen-Key)
    s = 1j * omega
    num = (A_ol * (s**2) * R1 * R2 * C1 * C2)
    den = (1 + s * (R1 * C1 + R2 * C1 + R2 * C2 * (1 - A_ol)) + (s**2) * R1 * R2 * C1 * C2)
    return num / den

# --- Función de barrido de frecuencia y Bode opcional ---

def analyze_filter(H_func, params, fmin=10, fmax=1e6, points=500, plot_bode=True, title="Filtro"):
    f = np.logspace(np.log10(fmin), np.log10(fmax), points)
    omega = 2 * np.pi * f
    H = np.array([H_func(*params, w) for w in omega])

    if plot_bode:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7,6))
        ax1.semilogx(f, db(H))
        ax1.set_ylabel("Magnitud (dB)")
        ax1.grid(True, which='both')
        ax2.semilogx(f, phase_deg(H))
        ax2.set_ylabel("Fase (°)")
        ax2.set_xlabel("Frecuencia (Hz)")
        ax2.grid(True, which='both')
        fig.suptitle("Diagrama de Bode - " + title)
        plt.tight_layout()
        plt.show()

    return f, H

# --- Ejemplos de uso (5 ejercicios) ---

def solve_examples(plot_bode=False):
    print("=== Respuesta en Frecuencia: 5 ejemplos ===\n")

    # Ejemplo 1: Filtro RC pasa bajos
    print("Ejemplo 1 — RC Pasa Bajos:")
    R, C = 1e3, 1e-6  # 1kΩ, 1µF
    fc = 1 / (2 * np.pi * R * C)
    print(f"Frecuencia de corte = {fc:.1f} Hz")
    analyze_filter(H_RC_lowpass, (R, C), title="RC Pasa Bajos", plot_bode=plot_bode)
    print()

    # Ejemplo 2: Filtro RC pasa altos
    print("Ejemplo 2 — RC Pasa Altos:")
    R, C = 1e3, 1e-6
    fc = 1 / (2 * np.pi * R * C)
    print(f"Frecuencia de corte = {fc:.1f} Hz")
    analyze_filter(H_RC_highpass, (R, C), title="RC Pasa Altos", plot_bode=plot_bode)
    print()

    # Ejemplo 3: Filtro RLC pasa banda
    print("Ejemplo 3 — RLC Pasa Banda:")
    R, L, C = 100, 10e-3, 1e-6
    f0 = 1 / (2 * np.pi * np.sqrt(L * C))
    print(f"Frecuencia central = {f0:.1f} Hz")
    analyze_filter(H_RLC_bandpass, (R, L, C), title="RLC Pasa Banda", plot_bode=plot_bode)
    print()

    # Ejemplo 4: Filtro activo Sallen–Key pasa bajos
    print("Ejemplo 4 — Sallen-Key Pasa Bajos (A_ol=1):")
    R1, R2, C1, C2, A_ol = 10e3, 10e3, 10e-9, 10e-9, 1
    analyze_filter(H_SallenKey_lowpass, (R1, R2, C1, C2, A_ol), fmax=1e5,
                   title="Sallen-Key Pasa Bajos", plot_bode=plot_bode)
    print()

    # Ejemplo 5: Filtro activo Sallen–Key pasa altos
    print("Ejemplo 5 — Sallen-Key Pasa Altos (A_ol=1):")
    R1, R2, C1, C2, A_ol = 10e3, 10e3, 10e-9, 10e-9, 1
    analyze_filter(H_SallenKey_highpass, (R1, R2, C1, C2, A_ol), fmax=1e5,
                   title="Sallen-Key Pasa Altos", plot_bode=plot_bode)
    print()

if __name__ == "__main__":
    solve_examples(plot_bode=True)